In [1]:
# NOTE: you should be using version 1.8.X of the Civis API client
import civis

### First, make some toy data

Let's suppose we want to make a model to determine whether an individual will likely be persuaded a particluar message. For each person in our (artificial) training data set, we have `age`, `gender`, and political `party` as features. We also have binary labels `msg` for whether or not the individual said they liked a message we showed them.

In [2]:
from io import StringIO
import pandas as pd
data = 'age,gender,party,msg\n20,M,other,1\n25,M,D,0\n30,F,D,0\n40,M,R,1\n50,F,D,0\n60,M,other,1\n65,F,R,0\n70,F,D,1\n80,M,R,1\n'

df = pd.read_csv(StringIO(data))

The first five rows in our training data look like:

In [3]:
df.head()

,age,gender,party,msg
0,20,M,other,1
1,25,M,D,0
2,30,F,D,0
3,40,M,R,1
4,50,F,D,0


### Next, let's build a model with CivisML

Using CivisML, we can train a random forest classifier to predict the probability that future individuals will also respond positively to the message.

In [4]:
mp = civis.ml.ModelPipeline(model='random_forest_classifier', dependent_variable='msg')
_ = mp.train(df)

In [5]:
mp.train_result_.train_job_id

11263303

You can monitor the status of your training job on Platform:

https://platform.civisanalytics.com/#/

Your model must finish training before you can deploy it.  

### We can deploy this model with two API calls

When the model has finished training, we can create a service for deployment by passing the training job ID (`mp.train_result_.train_job_id`) as an environment variable to our CivisML model deployment Docker image. You can optionally pass in the training run ID as well (`mp.train_result_.train_run_id`); if no such run ID is provided, the most recent run is used by default. 

In [6]:
client = civis.APIClient(resources='all')

resp1 = client.services.post(
    name='my_deployed_model',
    docker_image_name="civisanalytics/model-deployment",
    cpu=800,
    memory=8000, 
    environment_variables={'MODEL_JOB_ID': mp.train_result_.train_job_id, 
                           'DEBUG': 1}   # The "DEBUG" environment variable turns on verbose logging
)

The `DEBUG` environment variable ensures that our logs are verbose. You may want this turned off for models in production.

Once we have created our service, we can start the deployment with the following API call:

In [7]:
resp2 = client.services.post_deployments(resp1['id'])

It may take a minute or two for the deployment to start up, but once it does we can make requests from our model. You can monitor the logs of your deployment on Platform.

In [8]:
print("Platform page for your deployment: ")
print("https://platform.civisanalytics.com/#/services/{}".format(resp1['id']))

Platform page for your deployment: 
https://platform.civisanalytics.com/#/services/797


You can find the logs under the "Deployment History" link towards the upper right-hand side of the screen. When the logs say "_Application successfully changed to ready state_" you're ready to make calls.

### Let's make some calls to our model!

First, we'll need to get the model's URL and an access token. Once we have those, we can use python's `requests` library to make some simple RESTful calls. 

In [9]:
url = client.services.get(resp1['id'])['current_url']
print(url)

https://services--797--7ea03ac7669ac0a50bf15955--civis.shiny.services.civis.io


In [10]:
token_resp = client.services.post_tokens(resp1['id'], 'keiths_token')

We can get predictions from our model by passing covariates as query parameters to the `/predict` endpoint and making a GET call:

In [13]:
import requests

# Put the token in the header of the HTTP call
headers = {"Authorization": "Bearer {}".format(token_resp['token'])}
# Pass your model covariates as query parameters
pred_url = url + '/predict?age=30&gender=F&party=D'

# Make the GET call
resp = requests.get(pred_url, headers=headers)
print(resp.text)

{
  "msg_0": [
    0.936
  ], 
  "msg_1": [
    0.064
  ]
}



Look: a prediction! Predictions are returned as JSON dictionaries. 

Instead of passing our access token through the HTTP header, we can optionally pass it as a query parameter using the `civis_service_token` keyword:

In [14]:
pred_url = url + '/predict?age=30&gender=F&party=D&civis_service_token=' + token_resp['token']

# Make the GET call, no headers necessary
resp = requests.get(pred_url)
print(resp.text)

{
  "msg_0": [
    0.936
  ], 
  "msg_1": [
    0.064
  ]
}



### Model GUI

Go to your model's deployment page:

In [15]:
print("https://platform.civisanalytics.com/#/services/{}".format(resp1['id']))

https://platform.civisanalytics.com/#/services/797


Clicking on the "View Report" button towards the upper right-hand side of the screen will take you to a web form where you can enter model covariates and get predictions interactively. Try it!